<a href="https://colab.research.google.com/github/sarmientoj24/EE298/blob/master/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from keras.layers import Lambda, Input, Dense, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Activation, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.models import load_model

import numpy as nps
import matplotlib.pyplot as plt
import os
# Main cell to run
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Implement VAE here

class VAE():
  image_shape = (80, 60, 3)

  def __init__(self, load=False, encoder_h5=None, 
               decoder_h5=None):
    # Build encoder
    # Adopted from https://github.com/YongWookHa/VAE-Keras/blob/master/VAE.py

    if load:
      print("Loading model from h5...")
      self.load_model(encoder_h5, layer_ = 'encoder')
      self.load_model(decoder_h5, layer_ = 'decoder')

    else:
      print("Creating model architecture...")
      # self.encoder_inputs = Input(shape=self.image_shape)
      # filter_dim = 128
      # x = self.encoder_inputs
      # x = Conv2D(int(filter_dim/16), kernel_size=(2,2), strides=(2,2), padding='SAME')(x)
      # x = BatchNormalization()(x)
      # x = LeakyReLU(0.2)(x)
      # x = Conv2D(int(filter_dim/8), kernel_size=(2,2), strides=(2,2), padding='SAME')(x)
      # x = BatchNormalization()(x)
      # x = LeakyReLU(0.2)(x)
      # x = Conv2D(int(filter_dim/4), kernel_size=(2,2), strides=(2,2), padding='SAME')(x)
      # x = BatchNormalization()(x)
      # x = LeakyReLU(0.2)(x)
      # x = Conv2D(int(filter_dim/2), kernel_size=(2,2), strides=(2,2), padding='SAME')(x)
      # x = BatchNormalization()(x)
      # x = LeakyReLU(0.2)(x)
      # x = Conv2D(filter_dim, kernel_size=(2,2), strides=(2,2), padding='SAME')(x)
      # x = BatchNormalization()(x)
      # x = LeakyReLU(0.2)(x)
      # # x = GlobalAveragePooling2D()(x)

      # enc_shape = K.int_shape(x)
      # print(enc_shape)
      # # generate latent vector Q(z|X)
      # x = Flatten()(x)
      # x = Dense(64)(x)
      # x = LeakyReLU(0.2)(x)
      # z_mean = Dense(z_dim, name='z_mean')(x)
      # z_log_sigma = Dense(z_dim, name='z_log_sigma')(x)

      # # use reparameterization trick to push the sampling out as input
      # # note that "output_shape" isn't necessary with the TensorFlow backend
      # z = Lambda(self.sampling, output_shape=(z_dim,), name='z')([z_mean, z_log_sigma])

      # # instantiate encoder model
      # self.encoder = Model(self.encoder_inputs, [z_mean, z_log_sigma, z], name='encoder')
      # self.encoder.summary()
      # plot_model(self.encoder, to_file='vae_cnn_encoder.png', show_shapes=True)

      # # Decoder
      # self.latent_inputs = Input(shape=(z_dim,), name='z_sampling')
      # z = Dense(enc_shape[1] * enc_shape[2] * enc_shape[3], activation='relu')(self.latent_inputs)
      # z = Reshape((enc_shape[1], enc_shape[2], enc_shape[3]))(z)
      # z = Conv2DTranspose(int(filter_dim/2), kernel_size=(1, 2), strides=(2,1), padding='same')(z)
      # z = BatchNormalization()(z)
      # z = Activation('relu')(z)
      # z = Conv2DTranspose(int(filter_dim/4), kernel_size=(2,2), strides=(2,2), padding='same')(z)
      # z = BatchNormalization()(z)
      # z = Activation('relu')(z)
      # z = Conv2DTranspose(int(filter_dim/8), kernel_size=(2,1), strides=(2,2), padding='same')(z)
      # z = BatchNormalization()(z)
      # z = Activation('relu')(z)
      # z = Conv2DTranspose(int(filter_dim/16), kernel_size=(2,2), strides=(2,2), padding='same')(z)
      # z = BatchNormalization()(z)
      # z = Activation('relu')(z)
      # z = Conv2DTranspose(3, kernel_size=(2,2), strides=(2,2), padding='same')(z)
      # decoder_output = Activation('tanh')(z)

      # # instantiate decoder model
      # self.decoder = Model(self.latent_inputs, decoder_output, name='decoder')
      # self.decoder.summary()
      # plot_model(self.decoder, to_file='vae_cnn_decoder.png', show_shapes=True)

      # # instantiate VAE model
      # outputs = self.decoder(self.encoder(self.encoder_inputs)[2])
      # self.vae = Model(self.encoder_inputs, outputs, name='vae')
      # self.vae.summary()

      #### Meryl Version modified
      z_dim = 10
      kernel_size = 2
      latent_dim = 10
      filter_dims = [8, 16, 32]
      filter_decoder = [32, 16, 8]
      
      self.inputs = Input(shape=self.image_shape, name='encoder_input')
      x = self.inputs
      for i in range(2):
        filters = filter_dims[i]
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=2,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

      # shape info needed to build decoder model
      shape = K.int_shape(x)

      # generate latent vector Q(z|X)
      x = Flatten()(x)
      x = Dense(64, activation='relu')(x)
      z_mean = Dense(latent_dim, name='z_mean')(x)
      z_log_sigma = Dense(latent_dim, name='z_log_sigma')(x)

      # use reparameterization trick to push the sampling out as input
      # note that "output_shape" isn't necessary with the TensorFlow backend
      z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_sigma])

      # instantiate encoder model
      self.encoder = Model(self.inputs, [z_mean, z_log_sigma, z], name='encoder')
      self.encoder.summary()
      plot_model(self.encoder, to_file='vae_cnn_encoder.png', show_shapes=True)

      # build decoder model
      self.latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
      x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(self.latent_inputs)
      x = Reshape((shape[1], shape[2], shape[3]))(x)

      for i in range(2):
        filters = filter_decoder[i]
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

      outputs = Conv2DTranspose(filters=3,
                                kernel_size=kernel_size,
                                activation='tanh',      # Changed to tanh because data preparation was geared to be [-1, 1]
                                padding='same',
                                name='decoder_output')(x)

      # instantiate decoder model
      self.decoder = Model(self.latent_inputs, outputs, name='decoder')
      self.decoder.summary()
      plot_model(self.decoder, to_file='vae_cnn_decoder.png', show_shapes=True)

      # instantiate VAE model
      outputs = self.decoder(self.encoder(self.inputs)[2])
      self.vae = Model(self.inputs, outputs, name='vae')
      self.vae.summary()

      # Construct the Loss function
      # VAE loss = mse_loss or xent_loss + kl_loss

      reconstruction_loss = mse(K.flatten(self.inputs), K.flatten(outputs))
      # #reconstruction_loss = binary_crossentropy(K.flatten(self.encoder_inputs),
      # #                                           K.flatten(outputs))

      reconstruction_loss *= self.image_shape[0] * self.image_shape[1] * self.image_shape[2]
      kl_loss = 1 + (z_log_sigma ** 2) - K.square(z_mean) - K.exp(z_log_sigma ** 2)
      kl_loss = K.sum(kl_loss, axis=-1)
      kl_loss *= -0.5
      vae_loss = K.mean(reconstruction_loss + kl_loss)
      # recon = K.sum(K.binary_crossentropy(K.flatten(outputs), K.flatten(self.encoder_inputs)), axis=1)
      # kl = 0.5 * K.sum(K.exp(log_sigma) + K.square(mu) - 1. - log_sigma, axis=1)
      # vae_loss = recon + kl
      self.vae.add_loss(vae_loss)
      # self.vae.metrics_tensors.append(reconstruction_loss)
      # self.vae.metrics_names.append("val_loss")
      self.vae.compile(optimizer='adam')

  def get_summary(layer_='vae'):
    if layer_ == 'vae':
      self.vae.summary()

    elif layer == 'decoder':
      self.decoder.summary()

    else:
      self.encoder.summary()
  
  def load_model(self, h5_file, layer_):
    print("Loading h5 ", layer_)
    if layer_ == 'encoder':
      self.encoder = load_model(h5_file)
    else:
      self.decoder = load_model(h5_file)

  def __fit(self, epochs, batch_size, x_train, x_test=None):
    self.history = self.vae.fit(x_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1)
    self.vae.save('vae_mnist.h5')
    self.encoder.save('vae_encoder.h5')
    self.decoder.save('vae_decoder.h5')

  def get_history(self):
    return self.history
  
  def plot_loss_function(self):
    # Plot training & validation loss values
    plt.plot(self.history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()

  def get_encoder_model(self):
    return self.encoder
  
  def get_decoder_model(self):
    return self.decoder
  
  def get_vae_model(self):
    return self.vae
  
  # Sampling to be used in training
  def sampling(self, args):
    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(z_log_sigma / 2) * epsilon
    # return z_mean + K.exp(0.5 * z_log_var) * epsilon


In [0]:
# ImageEncoder
# After training VAE, create a new model based of the trained encoder
# for encoding the image samples
class ImageEncoder():
  input_shape = (80, 60, 3)

  def __init__(self, encoder, z_dim=10):
    self.encoder = Model(
        Input(shape=input_shape),
        encoder)
    self.epsilon = K.random_normal(shape=(z_dim, ))
  
  def __encode_sampled(self, input_image):
    z_mean, z_log_sigma, z = self.encoder.predict(input_image)
    # We only need z_mean and z_log_sigma
    return z_mean + K.exp(z_log_sigma / 2) * epsilon
    # return z_mean + K.exp(0.5 * z_log_var) * epsilon
  
  def __encode(self, input_image):
    z_mean, z_log_sigma, z = self.encoder.predict(input_image)
    return z_mean, z_log_sigma

In [0]:
#### HELPER FUNCTIONS
import cv2
import glob, os
import pandas as pd
from PIL import Image
import numpy as np

def load_image(infilename) :
    img = Image.open( infilename )
    data = np.asarray( img, dtype="int32" )
    print(img.size)
    return data, img.size

def read_imgs_to_np_from_folder(img_folder):
  cwd = os.getcwd()

  os.chdir(img_folder + '/')

  x = []
  for f in glob.glob("*.jpg"):
    img, shp = load_image(f) 

    # if shp == (60, 80):
    x.append(img)
    # else:
      #print("File not 80x60x3 ", f)
  
  os.chdir(cwd)
  # x = np.array(x)
  x = x / 255 * 2 - 1  # Scaled to -1 to 1 because it is tanh
  return np.array(X)

##### This function should be in the web-app (REMOVE once completed) ###########
import math
import numpy as np
import operator
# import pandas as pd

# decoder_loaded_model = load_model('decoder.h5')

def compute_log_likelihood(z_gen, mu_, log_sigma):
  # (zi−μi(x))22σi(x)2
  # Adopted here https://datascience.stackexchange.com/questions/64097/how-to-use-variational-autoencoders-%CE%BC-and-%CF%83-with-user-generated-z/64163#64163
  term_a = (z_gen - mu_) ** 2
  term_b = 2 * (log_sigma ** 2)
  term_c = -1 * (term_a / term_b)

  # −log(2π−−√σi(x))
  term_d = np.log(((2 * np.pi) ** 0.5) * (log_sigma))
  term_e = term_c - term_d

  likelihood = np.sum(term_e)
  return likelihood

def find_nearest(z_var_generated):
  # SQL Query get all of those needed and store to dataframe
  # Do Query here
  z_dim = len(z_var_generated)
  z_var_generated = np.array(z_var_generated) # Convert to numpy

  ids_ = sql_df['id'].to_list()
  df = sql_df[['mu_1', 'mu_2', 'mu_3', 'mu_4', 'mu_5', 
               'mu_6', 'mu_7', 'mu_8', 'mu_9', 'mu_10',
               'log_sigma_1', 'log_sigma_2', 'log_sigma_3', 'log_sigma_4', 'log_sigma_5',
               'log_sigma_6', 'log_sigma_7', 'log_sigma_8', 'log_sigma_9', 'log_sigma_10']]
  
  datapoints_encoded_in_db = df.to_numpy()
  rows = datapoints_encoded_in_db.shape[1]

  likelihoods = {}

  counter = 0
  for data in rows:
    mu_vector = data[:z_dim]
    sigma_vector = data[z_dim:z_dim*2]
    log_likelihood = compute_log_likelihood(z_var_generated,
                                            mu_vector, log_sigma_vector)
    likelihoods[id_s[counter]] = log_likelihood
    counter += 1

  # Sort dictionary by likelihood
  sorted_x = sorted(likelihoods.items(), key=operator.itemgetter(1))

  # return top three descending
  return sorted_x[0], sorted_x[1], sorted_x[2]

def decode_img(z_var_generated):
  img = decoder_loaded_model.predict(z_var_generated)
  return img

##########Functions above are to be used inside the web-app locally hosted #########3

# Function to use ENCODER and ENCODE datapoints to database
# Could be run on Colab or PC with GPU/Keras
import sqlite3
def encode_img_from_folder_to_db(folder, db, encoder):
  try:
    conn = sqlite3.connect(db)
  except Error as e:
    print(e)

  conn = self.create_connection(database)

  sql = '''
          SELECT id, image_id
          FROM appchemy_apparel_dummy
        '''

  products_df = pd.read_sql_query(sql, conn)
  image_ids = products_df['image_id'].to_list()

  cwd = os.getcwd()
  img_folder = folder + '/'

  cur = conn.cursor()
  os.chdir(img_folder)

  for index, image_id in enumerate(image_ids):
    img_name = str(image_id) + '.jpg'
    
    try:
        img = cv2.imread(img_name, mode='RGB')
        if img.shape() == (80, 60, 3):
          mu_, log_sigma_ = encoder.__encode(img)      # change if either to store z or mu and log_var

          # Update in db
          sql = '''
                UPDATE appchemy_apparel_dummy SET
                mu_1 = {}, mu_2 = {}, mu_3 = {}, mu_4 = {}, mu_5 = {}, 
                mu_6 = {}, mu_7 = {}, mu_8 = {}, mu_9 = {}, mu_10 = {},
                log_sigma_1 = {}, log_sigma_2 = {}, log_sigma_3 = {}, 
                log_sigma_4 = {}, log_sigma_5 = {}, log_sigma_6 = {}, 
                log_sigma_7 = {}, log_sigma_8 = {}, log_sigma_9 = {}, log_sigma_10 = {}
                WHERE id = {}
              '''.format(mu_[0], mu_[1], mu_[2], mu_[3], mu_[4], mu_[5], 
                         mu_[6], mu_[7], mu_[8], mu_[9], 
                         log_sigma_[0], log_sigma_[1], log_sigma_[2], log_sigma_[3], log_sigma_[4], 
                         log_sigma_[5], log_sigma_[6], log_sigma_[7], log_sigma_[8], log_sigma_[9],
                         image_id)
          cur.execute(sql)
          if index % 200 == 0:
            conn.commit()
          print("File not 80x60x3: ", img_name)
    except:
      print("Error reading file: ", img_name)
      continue

  conn.commit()
  conn.close()


In [0]:
# !chmod 600 '/content/drive/My Drive/ee298/dataset.tar.xz'
# !cp '/content/drive/My Drive/ee298/dataset.tar.xz' '/content/data_im/'

In [0]:
# !tar -xf '/content/data_im/dataset.tar.xz'

In [0]:
x_train = read_imgs_to_np_from_folder('/content/apparel_reduced')

(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(60, 80)
(